He was given an initial personal score of **S** when he enrolled, which changes every time he takes an exam: now comes the crazy part. He soon discovered that every of the **N** exams he has to take is assigned a mark **p**. Once he has chosen an exam, his score becomes equal to the mark **p**, and at the same time, the scoring system changes:

-  If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity **S-p**.
-  If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity **p-S**.


In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line, and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get.

**a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on because he wants to do paid counseling for other desperate students: in a recursive fashion, the helped helps the helpable.**

In [3]:
def solution1(S,grades):
    results=[]
    #define the function that returns the list with all the possible outcomes
    def function(grades,S,r):
        for i in range(len(grades)):
            #build the list with the new grades after taking one exam
            changed_grades=[x+(S-grades[i]) for x in grades[:i]+grades[i+1:]]
            if len(changed_grades)==1:
                r.append(changed_grades[0])
            else:
                function(changed_grades, grades[i],r)
        return r
    #find the highest possible final grade
    return(max(function(grades,S,[])))


**b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university!**


The time complexity of the provided code is O(N!) where N is the length of the list of initial grades. This is because the function `function(grades, S, r)` is recursively called for every element in the `grades` list, creating a new list for each call. 

The function essentially generates all permutations of the grades list. To be more specific, the function explores all possible outcomes of improving grades, which means it considers each possible sequence of exams a student could take. This is similar to generating all permutations of a list, which is known to have a time complexity of O(N!)
Here is a step-by-step breakdown of the time complexity:

1. The function is called once for each grade in the grades list, which gives us N recursive calls.
2. For each recursive call, the function creates a new list `changed_grades` that excludes one grade and adds (S - grade[i]) to each remaining grade. This operation has a time complexity of O(N) as it needs to iterate over the list.
3. Each recursive call then generates its own set of recursive calls, again one for each grade in the `changed_grades` list. This results in N! (N factorial) total function calls, since for each of the N grades there are (N-1) recursive calls, for each of those there are (N-2) calls, and so on down to 1.
4. The function then finds the maximum value in the results list, which requires iterating over the list once and has a time complexity of O(N).

Therefore, the overall time complexity is O(N!) due to the factorial number of recursive calls, each of which performs an operation that takes O(N) time. This is a worst-case scenario as it assumes that the recursive calls are not terminated early and all possible sequences of exams are explored.

**c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)**


To find an optimized code let's analyze the problem further:

1. Let's notice that it doesn't matter if the exam chosen next is "easier" or not because adding $S-p$ is equivalent to subtracting $p-S$. From now on we will only add $S-p$ when choosing a new exam

2. Let's see some examples of the process to get better understanding of what is happening.
    - initialize $S$ and $p=[p_1,p_2]$ and $p_1 \leq p_2$
        - randomly select $p_2$ so now: $S^1=p_2$ and $p^1=[p_1+S-p_2]=S^{final}$
        - randomly select $p_1$ so now: $S^1=p_1$ and $p^1=[p_2+S-p_1]=S^{final}$

        so the two possible final grades are $p_2+S-p_1$ or $p_1+S-p_2$ but since we are looking for the maximum we will select $p_2+S-p_1$ ($p_1 \leq p_2$)
    - initialize $S$ and $p=[p_1,p_2,p_3]$ and $p_1 \leq p_2 \leq p_3$
        - randomly select $p_3$ so now: $S^1=p_3$ and $p^1=[p_1+S-p_3, p_2+S-p_3]$
            - randomly select $p_2+S-p_3$ so now: $S^2=p_2+S-p_3$ and $p^2=[p_1+S-p_3+S^1-(p_2+S-p_3)]=[p_1+S-p_3+p_3-(p_2+S-p_3)]=[p_1+p_3-p_2]$
            - randomly select $p_1+S-p_3$ so now: $S^2=p_1+S-p_3$ and $p^2=[p_2+S-p_3+S^1-(p_1+S-p_3)]=[p_2+S-p_3+p_3-(p_1+S-p_3)]=[p_2+p_3-p_1]$

        choosing different paths will bring the same kind of result, where two grades are summed and one is subtracted and S doesn't come up in the final grade. So the maximum grade will of course be $p_2+p_3-p_1$
3. We noticed there is a difference between the results where the number of grades is even or odd. Let's propose a solution and let's try to demonstrate it.
$$S^{final, max} = \sum^n_{j=\frac{n-1}{2}+1}{p_j} - \sum^{\frac{n-1}{2}}_{j=1}{p_j} \quad {when\ n\ is\ odd}$$
$$S^{final, max} = S + \sum^n_{j=\frac{n}{2}+1}{p_j} - \sum^{\frac{n}{2}}_{j=1}{p_j} \quad {when\ n\ is\ even}$$

Let's show that if $n$ is odd then the final result does not depend from $S$. the algorithm has to make $n−1$ choices for the exams (the last is forced). Every 2 rounds of choices $S$ is added and subtracted (see previous example). $n-1$ is even so there is no $S$ in the final grade. Consequently, when $n$ is even, $S$ is added one last time.

Now we prove our optimized algorithm using strong induction on the length $n$ of the vector of exams.
- $n=1$ obvious
- $n=2,3$ shown above

By inductive hypothesis we have that the algorithm is right $\forall k<n$ and the same parity (we are actually doing two different problems).

Let's show when n is odd. In this case we can assume $S=0$ and $[p_1,p_2, ... , p_n]$ is in ascending order, We randomly choose $p_i$ as our first exam and adjust our list as $[p_1-p_i, ... , p_n-p_i]$ that now has $n-1$ grades ($n-1$ is even so we can't use the inductive hypotesis). Now we randomly choose $p_j-p_i$ as the next exam and adjust our list as $[p_1-p_j+p_i, ... , p_n-p_j+p_i]$ (calculations were shown before). Now we can use the hypotesis on the new list and say the maximum grade we can achieve is:
$$S^{final, max} = \sum^n_{k=\frac{n-1}{2}+1, k\not=i,j}{p_k-p_j+p_i} - \sum^{\frac{n-1}{2}}_{k=1, k\not=i,j}{p_k-p_j+p_i} $$ 
In order to make the values of the grades in the list the greatest possible (so that the final grade is as highest as possible), we should consider $i=n$ and $j=1$. In this case the grade is the greatest possible and, rewriting the last formula, is in the form:
$$S^{final, max} = \sum^n_{j=\frac{n-1}{2}+1}{p_j} - \sum^{\frac{n-1}{2}}_{j=1}{p_j}$$

which concludes our proof. The reasoning is similar for even values of $n$.

The code is simple:

In [4]:
def solution2(S, grades):
  n = len(grades)

  # Sort the grades in ascending order
  grades = sorted(grades)

  # Lower half of the grades summed:
  lower = sum([grades[i] for i in range(0, n // 2)])

  # Upper half of the grades summed:
  upper = sum([grades[i] for i in range(n // 2, n)]) 

  # following what we said earlier
  if n % 2 == 0:  
    return(S + upper - lower)
    
  if n % 2 == 1:
    return(upper - lower)

Let's analyze the time complexity of this code:

- Sorting the grades: The sorting operation has a time complexity of O(n log n), where n is the number of grades.

- Summing the lower half of grades: The sum operation involves iterating over the first half of the sorted list, which takes O(n/2) time in the worst case.

- Summing the upper half of grades: Similarly, the sum operation for the second half of the sorted list takes O(n/2) time in the worst case.

Therefore, the overall time complexity of the code is dominated by the sorting operation, and it is O(n log n).


**d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!**

This is the code that ChatGPT provided us

In [8]:
def solution_chatgpt(grades, current_score, memo):
    # Base case: If no exams left, return the current score
    if not grades:
        return current_score

    # Sort and convert to tuple for consistent memoization key
    grades_key = tuple(sorted(grades))

    # Check if this state has been computed before
    if (grades_key, current_score) in memo:
        return memo[(grades_key, current_score)]

    max_score = 0
    for i, grade in enumerate(grades):
        new_score = grade
        updated_grades = grades[:i] + grades[i+1:]

        if grade < current_score:
            updated_grades = [g + current_score - grade for g in updated_grades]
        else:
            updated_grades = [g - grade + current_score for g in updated_grades]

        max_score = max(max_score, solution_chatgpt(updated_grades, new_score, memo))

    memo[(grades_key, current_score)] = max_score
    return max_score

The time complexity of this recursive solution depends on the number of recursive calls and the work done within each call.

- Recursive Calls: In the worst case, the function explores all possible combinations of grades, making a recursive call for each combination. The number of recursive calls is exponential in the number of grades, but memoization helps avoid redundant calculations.

- Work within Each Call: For each recursive call, the function performs a constant amount of work, including sorting the grades, creating a tuple for memoization, and iterating through the grades. The dominant factor is the sorting operation, which has a time complexity of O(n log n), where n is the number of grades.

Considering both factors, the overall time complexity is O(n!), where n is the number of grades. Memoization significantly reduces the actual computation time by avoiding redundant calculations.
So it is somewhat better than solution1 but not solution2 which seems to be optimal.

**Solutions used on the examples in the homework** 

In [9]:
S=8
grades=[5,7,1]

print('First Solution: ',solution1(S, grades))
print('Second Solution: ',solution2(S, grades))
print('ChatGPT Solution: ',solution_chatgpt(grades,S,{}))

First Solution:  11
Second Solution:  11
ChatGPT Solution:  11


In [10]:
S=25
grades=[18, 24, 21, 32, 27]

print('First Solution: ',solution1(S, grades))
print('Second Solution: ',solution2(S, grades))
print('ChatGPT Solution: ',solution_chatgpt(grades,S,{}))

First Solution:  44
Second Solution:  44
ChatGPT Solution:  44


In [13]:
S=30
grades=[13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]


print('Second Solution: ',solution2(S, grades))
print('ChatGPT Solution: ',solution_chatgpt(grades,S,{}))

Second Solution:  205
ChatGPT Solution:  205


We are not showing the results for the first solution because it will take too much time